In [1]:
from pandas import read_csv
import numpy as np

In [2]:
def transform(df):
    
    df = df.drop(['passenger_count'], axis=1)
    
    # Create new features - dayofweek,hour,month,year
    df['dayofweek'] = df['pickup_datetime'].dt.dayofweek.astype('uint8')
    df['weekend'] = np.where(df.dayofweek < 5,0,1).astype('uint8')
    df = df.drop(['dayofweek'], axis=1)
    df['hour'] = df['pickup_datetime'].dt.hour.astype('uint8')
    #train_df['day'] = train_df['pickup_datetime'].dt.day.astype('uint8')
    #train_df['month'] = train_df['pickup_datetime'].dt.month.astype('uint8')
    #train_df['year'] = train_df['pickup_datetime'].dt.year.astype('uint16')
    #train_df['holiday'] = 

    # Create features for special rates
    df['eightpmto6am'] = np.where((df['hour'] >= 6) &
                                    (df['hour'] < 20),0,1).astype('uint8')

    # HOLIDAYS SHOULD BE INCLUDED IN THE CONDITION
    df['rushhour'] = np.where((df['hour'] >= 16) &
                                    (df['hour'] < 20) &
                                    (df['weekend'] == 0),1,0).astype('uint8')
    df = df.drop(['hour'], axis=1)
    
    # Add feature to saparete the periods with different fare rates
    df['period'] = np.where(df['pickup_datetime'] < '2012-09-01',0,1).astype('uint8')
    
    # Save memory
    df = df.drop(['pickup_datetime'], axis=1)
    #print(train_df.shape)
    #print('Maximum ride distance: %.1f' % train_df['dist'].max())
    return df

In [73]:
train_df = read_csv('train100K.csv', parse_dates=["pickup_datetime"],
                  infer_datetime_format=True)
train_df = train_df.drop(['key'], axis=1)
train_df = transform(train_df)
train_df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,weekend,eightpmto6am,rushhour,period
0,10.5,-73.978989,40.761620,-73.963959,40.770561,0,0,0,1
1,7.5,-73.980350,40.763070,-73.988682,40.753977,1,0,0,1
2,4.5,-74.002282,40.726577,-74.009582,40.723597,1,1,0,0
3,14.5,-73.932800,40.758900,-73.995000,40.736600,1,1,0,0
4,13.0,-73.975421,40.761071,-73.998444,40.724890,0,0,0,1


In [60]:
test_df = read_csv('test.csv', parse_dates=["pickup_datetime"],
                  infer_datetime_format=True)
test_df = transform(test_df)
test_df.head()

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,weekend,eightpmto6am,rushhour,period
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,0,0,0,1
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,0,0,0,1
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,0,0,0
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,1,0,1
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,1,0,1


In [65]:
sub = test_df[test_df.period==1]

In [66]:
sub.shape

(4338, 9)